In [328]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [329]:
TRAIN_CODE = pd.read_csv("../trimmed_data/SubwayStation-Code-trim.csv")
DVECT_EIGEN = [
    "다대포해수욕장노포","노포다대포해수욕장",
    "신평노포","노포신평",
    "장산호포","호포장산",
    "장산양산","양산장산",
    "전포양산","양산전포",
    "수영대저","대저수영",
    "미남안평","안평미남",
    "신평다대포해수욕장",
    "호포양산",
    "광안양산",
    "구명장산",
    "장산광안",
]

DAYTYPE = ['토요일', '일요일', '공휴일', '평일']
LoadPath_SubwayStation_ScheduleInterval_DayType = [f"../trimmed_data/SubwayStation-ScheduleInterval-{day}.json" for day in DAYTYPE]
SaveFilePath=[]

# `SubwayUniStation-IntervalTime.ipynb`
앞에서 만든 데이터를 통해서 한 역에서 배차 간격 구하기.

## 1. 데이터 불러오기

In [330]:
# RawData
db = pd.read_json(LoadPath_SubwayStation_ScheduleInterval_DayType[3],encoding="UTF-8")
db.head(3)

,TrainCode,LineNumber,DirectionVector,StationList,ArriveSchedule,DepartSchedule,DayType
0,1001,1,1,노포,308,308,평일
1,1001,1,1,범어사,309,310,평일
2,1001,1,1,남산,311,312,평일


In [331]:
DSVect_Index = db.iloc[:,2:6].groupby(by=["DirectionVector","StationList"]).indices.keys()
DSVect_Index = list(DSVect_Index)
# print(idx)
dVect_station = {}
for i in tqdm(DSVect_Index):
    # Weighted Sum
    _dat = db.iloc[:,2:6].groupby(by=["DirectionVector","StationList"]).get_group(i).iloc[:,2:]
    weight = pd.DataFrame({"Weight":[]})
    dVect_station[i]=_dat

dVect_station[DSVect_Index[6]].head(3)

  0%|          | 0/514 [00:00<?, ?it/s]

100%|██████████| 514/514 [00:04<00:00, 122.42it/s]


,ArriveSchedule,DepartSchedule
7079,382,0
7119,396,0
7159,408,0


## 2. Weight Trimming

In [332]:
df_weight = pd.read_csv("../raw_data/selected/부산교통공사_시간대별 승하차인원.csv",encoding="EUC-KR")
df_weight_SUM = df_weight.loc[:,["역번호","합계"]].groupby("역번호").sum()
target_columns = df_weight.columns[6:]

df_weight=df_weight.iloc[:,[0,*range(6,len(df_weight.columns))]].groupby("역번호").sum()
df_weight=df_weight.rename(columns={curr:new_name for curr,new_name in zip(target_columns,list(range(1,25)))})
# print(df_weight_SUM.index == df_weight.index) 왜 다 True 인데 안됨 ㅅㅂ?
for index in tqdm(df_weight_SUM.index):
    df_weight.loc[index,:] = df_weight.loc[index,:]/df_weight_SUM.loc[index,"합계"]
    

  0%|          | 0/112 [00:00<?, ?it/s]

/tmp/ipykernel_99478/1949101093.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6.510055163143897e-05' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_weight.loc[index,:] = df_weight.loc[index,:]/df_weight_SUM.loc[index,"합계"]
/tmp/ipykernel_99478/1949101093.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.403860845656166e-05' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_weight.loc[index,:] = df_weight.loc[index,:]/df_weight_SUM.loc[index,"합계"]
/tmp/ipykernel_99478/1949101093.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4.403860845656166e-05' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_weig

In [333]:
TRAIN_CODE = pd.read_csv("../trimmed_data/SubwayStation-Code-trim.csv")
TRAIN_CODE = TRAIN_CODE.set_index(keys="StationName")

print("ERROR OCCURED AT 종착역")
for i in tqdm(DSVect_Index):
    ## INNER LOOP VAR SETTING
    dVect, station_name = i
    stationCode = TRAIN_CODE.loc[
        station_name, "StationCode"
    ]  # stationName 을 staitonCode으로 바꿔줌
    hour = (
        dVect_station[i] // 60
    )  # minute 단위를 hour 단위로 : df_weight를 사용하기위해서.

    ## CONCAT
    # Concat을 위한 reset index - change target
    dVect_station[i] = dVect_station[i].reset_index()
    dVect_station[i] = dVect_station[i].iloc[:, 1:]
    
    #예외처리
    # Concat을 위한 reset index - add target
    
    try:
        ArriveWeight = df_weight.loc[stationCode, hour.loc[:, "ArriveSchedule"]]
        ArriveWeight = ArriveWeight.reset_index()
        ArriveWeight = ArriveWeight.iloc[:, 1]
        DepartWeight = df_weight.loc[stationCode, hour.loc[:, "DepartSchedule"]]
        DepartWeight = DepartWeight.reset_index()
        DepartWeight = DepartWeight.iloc[:, 1]
    except:
        print(f"[ERROR] : {DVECT_EIGEN[dVect]} - {station_name}",sep="")
        
    # Concat
    _weight = pd.DataFrame({"ArriveWeight": ArriveWeight, "DepartWeight": DepartWeight})
    dVect_station[i] = pd.concat([dVect_station[i], _weight], axis=1)

ERROR OCCURED AT 종착역


  2%|▏         | 10/514 [00:00<00:05, 95.68it/s]

[ERROR] : 다대포해수욕장노포 - 노포


 17%|█▋        | 85/514 [00:00<00:02, 190.35it/s]

[ERROR] : 노포다대포해수욕장 - 다대포해수욕장
[ERROR] : 신평노포 - 노포


...

[ERROR] : 전포양산 - 양산
[ERROR] : 양산전포 - 전포
[ERROR] : 수영대저 - 대저
[ERROR] : 수영대저 - 미남
[ERROR] : 수영대저 - 수영
[ERROR] : 대저수영 - 미남
[ERROR] : 대저수영 - 수영


 88%|████████▊ | 454/514 [00:02<00:00, 246.87it/s]

[ERROR] : 미남안평 - 미남
[ERROR] : 미남안평 - 안평
[ERROR] : 안평미남 - 미남
[ERROR] : 신평다대포해수욕장 - 다대포해수욕장
[ERROR] : 호포양산 - 양산


100%|██████████| 514/514 [00:02<00:00, 222.52it/s]

[ERROR] : 광안양산 - 양산
[ERROR] : 구명장산 - 수영
[ERROR] : 구명장산 - 장산
[ERROR] : 장산광안 - 광안
[ERROR] : 장산광안 - 수영


In [334]:
TRAIN_CODE = pd.read_csv("../trimmed_data/SubwayStation-Code-trim.csv")
print(TRAIN_CODE)
asdf={}
for i in tqdm(DSVect_Index):
    ArriveDiff = dVect_station[i]["ArriveSchedule"].diff()
    ArriveWeight = dVect_station[i]["ArriveWeight"]
    DepartDiff = dVect_station[i]["DepartSchedule"].diff()
    DepartWeight = dVect_station[i]["DepartWeight"]
    asdf[i] = (
        (ArriveDiff*ArriveWeight).sum()/ArriveWeight.sum(),
        (DepartDiff*DepartWeight).sum()/DepartWeight.sum()
    )

     StationCode StationName  LineCode
0             95     다대포해수욕장         1
1             96        다대포항         1
2             97          낫개         1
3             98         신장림         1
4             99          장림         1
..           ...         ...       ...
109          410          석대         4
110          411         영산대         4
111          412         윗반송         4
112          413          고촌         4
113          414          안평         4

[114 rows x 3 columns]


  0%|          | 0/514 [00:00<?, ?it/s]

100%|██████████| 514/514 [00:00<00:00, 1480.13it/s]


In [335]:
asdf

{(0, '괴정'): (6.044520006349198, 6.05172564892953),
 (0, '교대'): (5.990767689379536, 5.990241058553255),
 (0, '구서'): (6.112879809203258, 6.118175581101701),
 (0, '남산'): (6.127180406385333, 6.12767557966527),
 (0, '남포'): (6.061563387835182, 6.056662139357808),
 (0, '낫개'): (6.045420318385101, 6.037553066503068),
 (0, '노포'): (6.199858456887805, 0.0),
 (0, '다대포항'): (6.064702758926688, 6.062306042981311),
 (0, '다대포해수욕장'): (6.0743391555744495, 6.0743391555744495),
 (0, '당리'): (6.0369151129614504, 6.02370609573281),
 (0, '대티'): (6.047067466530023, 6.03767263805857),
 (0, '동대신'): (6.033576714639546, 6.041638905910536),
 (0, '동래'): (6.6877084094105905, 7.202147517826789),
 (0, '동매'): (5.95072408999794, 5.949556683364923),
 (0, '두실'): (6.111562208267618, 6.117610167372163),
 (0, '명륜'): (6.0725372980296, 6.068185438766304),
 (0, '범내골'): (5.871272496433476, 5.889038939618017),
 (0, '범어사'): (6.148517050699123, 6.141141050409483),
 (0, '범일'): (6.017594496683796, 6.005455805567556),
 (0, '부산대'): (6.095